In [ ]:
!pip install -e ../../../../.. --quiet

In [2]:
!pip install python-dotenv

# Create .env file containing the following line:
# LASTMILE_API_TOKEN=<your-token>
# You can get your key from the "API Token" section of https://lastmileai.dev/settings?page=tokens 
import dotenv
import os
dotenv.load_dotenv()
lastmile_api_token = os.getenv("LASTMILE_API_TOKEN")
assert lastmile_api_token is not None

# Pretty hacky way of locally installing the lastmile_eval package lol
os.getcwd()


'/Users/jonathan/Projects/eval/src/lastmile_eval/examples/rag_debugger/tracing'

In [7]:
import requests
from requests import Response

# First create a project Id
project_name = "My Fancy Project Name"

lastmile_evaluation_projects_endpoint = (
    "https://lastmileai.dev/api/evaluation_projects/create"
)
response: Response = requests.post(
    lastmile_evaluation_projects_endpoint,
    headers={"Authorization": f"Bearer {lastmile_api_token}"},
    json={
        "name": project_name,
        # "testCases": [{"query": query_str}],
    },
    timeout=60,  # TODO: Remove hardcoding
)

# Save this projectId and pass it into the other two endpoints below
print("Results from evaluation_projects create endpoint:")
print(response.json())

Results from evaluation_projects create endpoint:
{'id': 'clv5l7exn002upbwoekigdvxx', 'createdAt': '2024-04-18T18:40:47.196Z', 'updatedAt': '2024-04-18T18:40:47.196Z', 'name': 'My Fancy Project Name', 'metadata': None, 'creatorId': 'clp1m7n3l0062qpqnd4nyabbl', 'organizationId': None, 'visibility': 'PRIVATE', 'active': True}


In [4]:
# Now go to trace/sdk.py file and change the hardcoded project_id to the one 
# you just created. 

# For sanity check, re-install pip package, then restart 
# ipynb kernel and re-run from this point onwards
!pip install -e ../../../../.. --quiet

In [3]:
from openinference.semconv.trace import (
    OpenInferenceSpanKindValues,
    SpanAttributes,
)
from opentelemetry import trace as trace_api
from opentelemetry.trace import StatusCode

from lastmile_eval.rag.debugger.api import (
    LastMileTracer,
    QueryReceived,
    ContextRetrieved,
    PromptResolved,
    LLMOutputReceived,
)

# TODO: Add these both to the API library instead of relying on SDK
from lastmile_eval.rag.debugger.tracing import (
    get_lastmile_tracer,
)

# Define a LastMileTracer, which contains the same API interface as an
# OpenTelemetry tracer
assert lastmile_api_token is not None
tracer: LastMileTracer = get_lastmile_tracer(
    tracer_name="my-tracer",
    lastmile_api_token=lastmile_api_token,
    initial_params={"global_param": "This param key appear in all traces"},
)

# We do not have an existing trace running so this parameter will be registered
# to all subsequent traces
tracer.register_param(
    "overriden_by_trace_example", "This is the global value!"
)


def _convert_int_id_to_hex_str(int_id: int) -> str:
    """
    Helper function to convert an integer id to a hex string. This is
    needed because Jaeger does trace and span queries using hex strings
    instead of integer values.

    Ex: int_id = 123456789 -> hex value = 0x75BCD15 --> str = "75BCD15"

    @param int_id (int): The integer id to convert to a hex string

    @return str: The hex string representation of the integer id
    """
    return str(hex(int_id)).split("x")[1]


# Same tracer functionality for logging spans as usual for OpenTelemetry
@tracer.start_as_current_span(
    "root-span"  # Span finishes automatically when retrieval_function ends
)
def generate_trace_data(count: int, resulting_trace_ids: list[str]) -> None:
    root_span = trace_api.get_current_span()
    trace_id = _convert_int_id_to_hex_str(
        root_span.get_span_context().trace_id
    )
    root_span.set_attribute(
        SpanAttributes.OPENINFERENCE_SPAN_KIND,
        OpenInferenceSpanKindValues.AGENT.value,
    )
    # print("We are in the root span now ")

    # Can also use embedded with-blocks instead of decorators around methods
    with tracer.start_as_current_span("child-span") as child_span:
        child_span.set_attribute(
            SpanAttributes.OPENINFERENCE_SPAN_KIND,
            OpenInferenceSpanKindValues.CHAIN.value,
        )

        # Example of logging an event
        query = QueryReceived(query="Query #{count}")
        tracer.mark_rag_query_trace_event(
            event=query,
            # test_set_id=str(1234),
            # indexing_trace_id=str(5678),
        )

        context = ContextRetrieved(
            context=["Run #{count}, context 1", "Run #{count}, context 2"]
        )
        tracer.mark_rag_query_trace_event(
            event=context,
            # test_set_id=str(1234),
            # indexing_trace_id=str(5678),
        )

        tracer.mark_rag_query_trace_event(
            event=PromptResolved(
                fully_resolved_prompt=f"User Query: {query.query}\n\nContext: {context.context}"
            ),
            # test_set_id=str(1234),
            # indexing_trace_id=str(5678),
        )

        tracer.mark_rag_query_trace_event(
            event=LLMOutputReceived(llm_output=f"Output #{count}"),
            # test_set_id=str(1234),
            # indexing_trace_id=str(5678),
        )

        # print(log_result)

        # This parameter has the same key as something that's already stored
        # at the tracer level (one level above current trace). We will
        # overwrite the K-V pair for the trace-specific params, but when we
        # create a new trace the old value will remain
        tracer.register_param(
            "overriden_by_trace_example",
            f"Overriden in run #{count} for traceId '{trace_id}'",
        )

        # This parameter has the same key as something that's already stored
        # at the tracer level (one level above current trace). We will
        # overwrite the K-V pair for the trace-specific params, but when we
        # create a new trace the old value will remain
        tracer.register_param("local_param", f"Local param #{count}")

        child_span.set_status(StatusCode.OK)

    trace_id = _convert_int_id_to_hex_str(
        root_span.get_span_context().trace_id
    )
    resulting_trace_ids.append(trace_id)


In [6]:
# Generate 20 queries to seed DB
trace_ids: list[str] = []
for i in range(20):
    generate_trace_data(count=i+1, resulting_trace_ids=trace_ids)
# print(f"{trace_ids=}")

2024-04-19 16:18:00,092 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-04-19 16:18:00,232 - https://lastmileai.dev:443 "POST /api/rag_query_traces/create HTTP/1.1" 200 829
2024-04-19 16:18:00,235 - Starting new HTTPS connection (1): lastmileai.dev:443


trace_id_before=None
trace_id_after='a40e1c970fe5aa6df86a366af4d8936f'
trace_id_before='a40e1c970fe5aa6df86a366af4d8936f'
trace_id_after='a40e1c970fe5aa6df86a366af4d8936f'
Results from rag traces create endpoint:
{'id': 'clv744aem003dpeushnoe19cl', 'createdAt': '2024-04-19T20:18:00.238Z', 'updatedAt': '2024-04-19T20:18:00.238Z', 'paramSet': {'local_param': 'Local param #1', 'global_param': 'This param key appear in all traces', 'overriden_by_trace_example': "Overriden in run #1 for traceId 'a40e1c970fe5aa6df86a366af4d8936f'"}, 'query': '{"query":"Query #{count}"}', 'context': '{"context":["Run #{count}, context 1","Run #{count}, context 2"]}', 'fullyResolvedPrompt': '{"fully_resolved_prompt":"User Query: Query #{count}\\n\\nContext: [\'Run #{count}, context 1\', \'Run #{count}, context 2\']"}', 'output': '{"llm_output":"Output #1"}', 'metadata': None, 'traceId': 'a40e1c970fe5aa6df86a366af4d8936f', 'ragIngestionTraceId': None, 'creatorId': 'clnz1fbdn005xpei7ekv837tb', 'projectId': None,

2024-04-19 16:18:00,346 - https://lastmileai.dev:443 "POST /api/rag_query_traces/create HTTP/1.1" 200 829
2024-04-19 16:18:00,350 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-04-19 16:18:00,454 - https://lastmileai.dev:443 "POST /api/rag_query_traces/create HTTP/1.1" 200 829
2024-04-19 16:18:00,458 - Starting new HTTPS connection (1): lastmileai.dev:443


Results from rag traces create endpoint:
{'id': 'clv744aht00a9qpy4w2zduyy8', 'createdAt': '2024-04-19T20:18:00.353Z', 'updatedAt': '2024-04-19T20:18:00.353Z', 'paramSet': {'local_param': 'Local param #2', 'global_param': 'This param key appear in all traces', 'overriden_by_trace_example': "Overriden in run #2 for traceId 'e70f51331f6020705ae0a9586ff50f82'"}, 'query': '{"query":"Query #{count}"}', 'context': '{"context":["Run #{count}, context 1","Run #{count}, context 2"]}', 'fullyResolvedPrompt': '{"fully_resolved_prompt":"User Query: Query #{count}\\n\\nContext: [\'Run #{count}, context 1\', \'Run #{count}, context 2\']"}', 'output': '{"llm_output":"Output #2"}', 'metadata': None, 'traceId': 'e70f51331f6020705ae0a9586ff50f82', 'ragIngestionTraceId': None, 'creatorId': 'clnz1fbdn005xpei7ekv837tb', 'projectId': None, 'organizationId': None, 'visibility': 'MEMBER', 'active': True}
trace_id_before=None
trace_id_after='e292ebb4c5b32e36468ba2beff10d6e3'
trace_id_before='e292ebb4c5b32e36468

2024-04-19 16:18:00,600 - https://lastmileai.dev:443 "POST /api/rag_query_traces/create HTTP/1.1" 200 829
2024-04-19 16:18:00,604 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-04-19 16:18:00,732 - https://lastmileai.dev:443 "POST /api/rag_query_traces/create HTTP/1.1" 200 829
2024-04-19 16:18:00,736 - Starting new HTTPS connection (1): lastmileai.dev:443


...

2024-04-19 16:18:02,309 - https://lastmileai.dev:443 "POST /api/rag_query_traces/create HTTP/1.1" 200 830
2024-04-19 16:18:02,313 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-04-19 16:18:02,439 - https://lastmileai.dev:443 "POST /api/rag_query_traces/create HTTP/1.1" 200 830
2024-04-19 16:18:02,442 - Starting new HTTPS connection (1): lastmileai.dev:443


Results from rag traces create endpoint:
{'id': 'clv744c0e008kpbhcqbogl1h1', 'createdAt': '2024-04-19T20:18:02.319Z', 'updatedAt': '2024-04-19T20:18:02.319Z', 'paramSet': {'local_param': 'Local param #18', 'global_param': 'This param key appear in all traces', 'overriden_by_trace_example': "Overriden in run #18 for traceId '6dd0cf3fa8465c79eac684353025ee6'"}, 'query': '{"query":"Query #{count}"}', 'context': '{"context":["Run #{count}, context 1","Run #{count}, context 2"]}', 'fullyResolvedPrompt': '{"fully_resolved_prompt":"User Query: Query #{count}\\n\\nContext: [\'Run #{count}, context 1\', \'Run #{count}, context 2\']"}', 'output': '{"llm_output":"Output #18"}', 'metadata': None, 'traceId': '6dd0cf3fa8465c79eac684353025ee6', 'ragIngestionTraceId': None, 'creatorId': 'clnz1fbdn005xpei7ekv837tb', 'projectId': None, 'organizationId': None, 'visibility': 'MEMBER', 'active': True}
trace_id_before=None
trace_id_after='26bc25356b8cb9f637f22e7f27d8f91'
trace_id_before='26bc25356b8cb9f637f

2024-04-19 16:18:02,554 - https://lastmileai.dev:443 "POST /api/rag_query_traces/create HTTP/1.1" 200 832


Results from rag traces create endpoint:
{'id': 'clv744c76008lpbhcfgkpzmvn', 'createdAt': '2024-04-19T20:18:02.563Z', 'updatedAt': '2024-04-19T20:18:02.563Z', 'paramSet': {'local_param': 'Local param #20', 'global_param': 'This param key appear in all traces', 'overriden_by_trace_example': "Overriden in run #20 for traceId 'da58981d43395daf19f27bbb6dcf28d4'"}, 'query': '{"query":"Query #{count}"}', 'context': '{"context":["Run #{count}, context 1","Run #{count}, context 2"]}', 'fullyResolvedPrompt': '{"fully_resolved_prompt":"User Query: Query #{count}\\n\\nContext: [\'Run #{count}, context 1\', \'Run #{count}, context 2\']"}', 'output': '{"llm_output":"Output #20"}', 'metadata': None, 'traceId': 'da58981d43395daf19f27bbb6dcf28d4', 'ragIngestionTraceId': None, 'creatorId': 'clnz1fbdn005xpei7ekv837tb', 'projectId': None, 'organizationId': None, 'visibility': 'MEMBER', 'active': True}


2024-04-19 16:18:04,472 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10


In [7]:
import requests

entity_id = "clv743t8h003bpeusmoes4nnv"
print(lastmile_api_token)
response = requests.get(
    f"https://lastmileai.dev/api/rag_query_traces/read?id={entity_id}",
    headers={"Authorization": f"Bearer {lastmile_api_token}"},
    timeout=60,  # TODO: Remove hardcoding
)
print(response.json())

2024-04-19 16:18:04,964 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-04-19 16:18:05,079 - https://lastmileai.dev:443 "GET /api/rag_query_traces/read?id=clv743t8h003bpeusmoes4nnv HTTP/1.1" 200 827


eyJhbGciOiJkaXIiLCJlbmMiOiJBMjU2R0NNIn0..i4MtL9O76DKs-6Vg.wiKue2CUsvsau5jDwIfDjc-PMAhcFYpM43t_HbL_0Q2j79HGDlBkRXG71sXBTpFzNNOq5SsrA06akX-l6LQWPQbUKZ-MVz3b-ceKuKlA0uH-xJNr6HVXHcVBfLx7SHeLinK8W_eYN4MeJOJ8n8T4ILnpy7Li19hZRouMVLC_HezlpGQPG4pQfYkoq1BLWfecgFErtlb7QmWnwujc6N4pvqlahpLJw4DBiJOicGJWHHd52yg40Vu7n_J9C-y02X8.YFl27avAV7Mr_WBoQmcvEg
{'id': 'clv743t8h003bpeusmoes4nnv', 'createdAt': '2024-04-19T20:17:37.985Z', 'updatedAt': '2024-04-19T20:17:37.985Z', 'paramSet': {'local_param': 'Local param #1', 'global_param': 'This param key appear in all traces', 'overriden_by_trace_example': "Overriden in run #1 for traceId 'a20c9c67f417e68501d2c2c72589ed6'"}, 'query': '{"query":"Query #{count}"}', 'context': '{"context":["Run #{count}, context 1","Run #{count}, context 2"]}', 'fullyResolvedPrompt': '{"fully_resolved_prompt":"User Query: Query #{count}\\n\\nContext: [\'Run #{count}, context 1\', \'Run #{count}, context 2\']"}', 'output': '{"llm_output":"Output #1"}', 'metadata': None, 'traceId': 'a